# V6: bench5

In [1]:
from parase_input_package.generate_output import *
from parase_input_package.parase_input import *
from parase_input_package.plot_output import *
filepath_out='../output/bench5.router'
netlist_file_path = '../benchmark/bench5.nl'
gridfile_path  = '../benchmark/bench5.grid'
nets,net_num = parse_netlist(netlist_file_path)
rows,columns,bend_penalty,via_penalty,layer1_grid_original,layer2_grid_original = parse_gridfile(gridfile_path)

bend_penalty: 10 
 via_penalty: 20
size of layer1: (127, 317)
size of layer2: (127, 317)


In [2]:
# plot problem 根据grid文件可视化障碍物和每个格子的cost，同时标出source和target，可以显示两层
plot_problem("../output/bench5_problem.jpg",columns,rows,layer1_grid_original,layer2_grid_original,nets)

In [3]:
def reconstruct_path(source, target, parents):
    path = []
    current = target
    while current != source:
        path.append(current)
        current = parents[current]
    path.append(source)
    path.reverse()
    return path

def reconstruct_path_tmp(source, target_tmp, parents):
    path_tmp = []
    current = target_tmp
    while current != source and len(path_tmp)<= 3:
        path_tmp.append(current)
        current = parents[current]
    path_tmp.append(source)
    path_tmp.reverse()
    return path_tmp

def mark_path_on_grid(layer1_grid,layer2_grid, path):
    if path:               
        for cell in path:
            x, y, layer = cell
            if  (layer == 1):
                layer1_grid[x][y] = -1
            if  (layer == 2):
                layer2_grid[x][y] = -1

def get_cell_cost(layer_grid, cell,path_tmp,bend_penalty,via_penalty):  
    x, y, layer = cell
    cell_cost = 1  # 默认的单元代价
    if layer_grid[x][y] == -1:
        cell_cost = float('inf')  # -1表示无法通过的细胞
    elif layer_grid[x][y] != 1:
        cell_cost = layer_grid[x][y]  # 非单元代价
    if len(path_tmp) >= 2:
        prev_cell = path_tmp[-2]
        prev_x, prev_y, prev_layer = prev_cell
        if layer != prev_layer:
            cell_cost += via_penalty
        elif (prev_x != x and prev_y != y):  
            cell_cost += bend_penalty
    return cell_cost

def expand_source_to_target(rows, columns, layer1_grid,layer2_grid, source, target,bend_penalty,via_penalty):
    wavefront = {}
    visited = set()
    parents = {}
    costs = {}  # Store the cumulative costs for each cell
    costs_targe = {} 
    
    
    source_tuple = (source['x'], source['y'], source['layer'])
    target_tuple = (target['x'], target['y'], target['layer'])
    
    wavefront[source_tuple] = 0
    costs[source_tuple] = 1  # Initial cost for the source cell is 1

    while wavefront:
        # get lowest cost cell on a wavefront structure
        current_cell = sorted(wavefront.items(),key=lambda s:int(s[1]))[0][0]

        if current_cell == target_tuple:
            path = reconstruct_path(source_tuple, target_tuple, parents)
            return path,costs[current_cell]

        neighbors = get_neighbors(rows, columns, current_cell)

        for neighbor in neighbors:
            neighbor_tuple = (neighbor['x'], neighbor['y'], neighbor['layer'])
            if neighbor_tuple not in visited:
                path_tmp= reconstruct_path_tmp(source_tuple, current_cell, parents)
                # Calculate the cost to reach the neighbor cell
                if(neighbor_tuple[2]== 1):
                  cost = costs[current_cell] + get_cell_cost(layer1_grid, neighbor_tuple,path_tmp,bend_penalty,via_penalty)
                if(neighbor_tuple[2]== 2):
                  cost = costs[current_cell] + get_cell_cost(layer2_grid, neighbor_tuple,path_tmp,bend_penalty,via_penalty)
                # ignore blocks
                if cost!= np.inf:
                    if neighbor_tuple not in wavefront.keys() or costs[neighbor_tuple] > cost:
                        costs[neighbor_tuple] = cost
                        parents[neighbor_tuple] = current_cell
                    if neighbor_tuple not in wavefront.keys():
                        # add cell N to waveform, indexed by pathcost
                        costs_targe = cost + abs(neighbor_tuple[0] - target_tuple[0] ) + abs(neighbor_tuple[1] - target_tuple[1] ) 
                        wavefront[neighbor_tuple]=costs_targe         

        visited.add(current_cell)    
        del wavefront[current_cell]                  
    return None,None

def get_neighbors(rows, columns, cell):  
    x, y, layer = cell
    neighbors = []
    if layer == 1: 
        if x < columns - 1:
            neighbors.append({'x': x + 1, 'y': y, 'layer': layer})
        if x > 0:
            neighbors.append({'x': x - 1, 'y': y, 'layer': layer})  
        neighbors.append({'x': x, 'y': y, 'layer': 3-layer})    
    elif layer == 2: 
        if y < rows - 1:
            neighbors.append({'x': x, 'y': y + 1, 'layer': layer})
        if y > 0:
            neighbors.append({'x': x, 'y': y - 1, 'layer': layer})  
        neighbors.append({'x': x, 'y': y, 'layer': 3-layer})                                       
    return neighbors

def true_two_layer_router(rows, columns, layer1_grid,layer2_grid, nets,bend_penalty,via_penalty):
    nets.sort(key=lambda s: (abs(s["pin1"]['x']-s["pin2"]['x'])+abs(s["pin1"]['y']-s["pin2"]['y'])),reverse = False)
    routing_table = {}
    costs_table = {}
    for net in nets:
        net_id = net['net_id']
        pin1 = net['pin1']
        pin2 = net['pin2']
        # 防止布线在后续的pin上，先将所有的pin标记为-1；
        if pin1['layer'] == 1:
            layer1_grid[pin1['x']][pin1['y']] = -1
        if pin1['layer'] == 2:
            layer2_grid[pin1['x']][pin1['y']] = -1
        if pin2['layer'] == 1:
            layer1_grid[pin2['x']][pin2['y']] = -1
        if pin2['layer'] == 2:
            layer2_grid[pin2['x']][pin2['y']] = -1
           
    for net in nets:
        net_id = net['net_id']
        pin1 = net['pin1']
        pin2 = net['pin2']
        if pin1['layer'] == 1:
            layer1_grid[pin1['x']][pin1['y']] = 1
        if pin1['layer'] == 2:
            layer2_grid[pin1['x']][pin1['y']] = 1
        if pin2['layer'] == 1:
            layer1_grid[pin2['x']][pin2['y']] = 1
        if pin2['layer'] == 2:
            layer2_grid[pin2['x']][pin2['y']] = 1
        print('Routing net:',net_id)
        path,costs = expand_source_to_target(rows, columns, layer1_grid,layer2_grid, pin1, pin2,bend_penalty,via_penalty) 
        if path is not None:
            mark_path_on_grid(layer1_grid,layer2_grid,path)
            routing_table[net_id] = path
            costs_table[net_id] = costs
    routing_table = dict(sorted(routing_table.items(),key=lambda x:x[0]))
    costs_table = dict(sorted(costs_table.items(),key=lambda x:x[0]))
    return routing_table,costs_table

In [4]:
layer1_grid = layer1_grid_original.copy()
layer2_grid = layer2_grid_original.copy()
routing_table,costs_table=true_two_layer_router(rows, columns, layer1_grid.T,layer2_grid.T, nets,bend_penalty,via_penalty)
plot_path('../output/bench5.jpg',columns=columns,rows=rows,grid1=layer1_grid_original,grid2=layer2_grid_original,path_dict=routing_table) 
generate_output_file(filepath_out,net_num,routing_table)

Routing net: 121
Routing net: 123
Routing net: 68
Routing net: 72
Routing net: 75
Routing net: 92
Routing net: 13
Routing net: 62
Routing net: 35
Routing net: 87
Routing net: 76
Routing net: 11
Routing net: 40
Routing net: 82
Routing net: 22
Routing net: 37
Routing net: 5
Routing net: 10
Routing net: 48
Routing net: 81
Routing net: 114
Routing net: 1
Routing net: 19
Routing net: 23
Routing net: 49
Routing net: 125
Routing net: 122
Routing net: 100
Routing net: 69
Routing net: 31
Routing net: 12
Routing net: 32
Routing net: 42
Routing net: 51
Routing net: 66
Routing net: 106
Routing net: 64
Routing net: 89
Routing net: 96
Routing net: 104
Routing net: 109
Routing net: 8
Routing net: 24
Routing net: 63
Routing net: 47
Routing net: 70
Routing net: 107
Routing net: 54
Routing net: 61
Routing net: 120
Routing net: 124
Routing net: 56
Routing net: 73
Routing net: 111
Routing net: 2
Routing net: 44
Routing net: 59
Routing net: 99
Routing net: 98
Routing net: 115
Routing net: 38
Routing net: 4

In [5]:
costs_table


{1: 103,
 2: 186,
 3: 197,
 4: 233,
 5: 61,
 6: 236,
 7: 216,
 8: 176,
 9: 206,
 10: 168,
 11: 160,
 12: 178,
 13: 154,
 14: 224,
 15: 255,
 16: 183,
 17: 321,
 18: 212,
 19: 103,
 20: 212,
 21: 209,
 22: 164,
 23: 103,
 24: 178,
 25: 224,
 26: 224,
 27: 223,
 28: 200,
 29: 210,
 30: 197,
 31: 175,
 32: 176,
 33: 173,
 34: 243,
 35: 156,
 36: 224,
 37: 164,
 38: 172,
 39: 200,
 40: 161,
 41: 188,
 42: 178,
 43: 172,
 44: 186,
 45: 193,
 46: 214,
 47: 177,
 48: 82,
 49: 148,
 50: 220,
 51: 174,
 52: 196,
 53: 202,
 54: 181,
 55: 239,
 56: 186,
 57: 218,
 58: 198,
 59: 190,
 60: 223,
 61: 184,
 62: 9,
 63: 156,
 64: 177,
 65: 212,
 66: 176,
 67: 163,
 68: 152,
 69: 172,
 70: 178,
 71: 194,
 72: 152,
 73: 122,
 74: 109,
 75: 152,
 76: 159,
 77: 194,
 78: 199,
 79: 207,
 80: 218,
 81: 82,
 82: 163,
 83: 211,
 84: 192,
 85: 213,
 86: 217,
 87: 156,
 88: 174,
 89: 175,
 90: 205,
 91: 243,
 92: 152,
 93: 205,
 94: 203,
 95: 324,
 96: 177,
 97: 211,
 98: 172,
 99: 166,
 100: 108,
 101: 233,
 1

In [6]:
sum(costs_table.values()) 
#单一方向：由难到易15372，完成124
#四个方向，由易到难12310，完成99
#单一方向，由易到难21982，完成125

23266

In [7]:
cnt = 0
for item in routing_table.keys():
    if routing_table[item]:
        cnt += 1
cnt

128